# How PyPy can help High Performance Computing

## Short bio
bla bla bla

# How many of you use Python?

# How many have ever had performance problems?

# Why do you use Python, then?

# Python strong points

* Simplicity
* Lots of libraries
* Ecosystem

<img width="100%" src="tracing-phases.svg" />